In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Programa para encontrar empresas rurais nas quais os deputados federais são sócios ou administradores
#

In [ ]:
import pandas as pd
import unidecode

Função para retirar acentuação 

In [2]:
def f(str):
    return (unidecode.unidecode(str))

Acessa a lista de nomes de deputados eleitos em 2014

In [5]:
eleitos_d = pd.read_excel('/home/reinaldo/Documentos/Code/RB2/candidatos/repo/resultados/dep_eleitos_2014_unidos.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 
                                                                                                                  'cpf': lambda x: str(x),  
                                                                                                                  'deputado_id': lambda x: str(x)})

In [6]:
eleitos_d["nome_completo"] = eleitos_d["nome_completo"].apply(f)

Acessa informações da Receita Federal

O programa que acessa esses dados está em: https://github.com/reichaves/curso_knight_python_2017/blob/master/roboempresa.ipynb

In [8]:
socios_brasil = pd.read_csv("/home/reinaldo/Documentos/Code/intercept/seguranca/repo/socios_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [7]:
deputados_socios = pd.merge(eleitos_d, socios_brasil, left_on='nome_completo', right_on='socio')

In [7]:
empresas_brasil = pd.read_csv("/home/reinaldo/Documentos/Code/intercept/seguranca/repo/empresas_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [9]:
deputados_socios_empresas = pd.merge(deputados_socios, empresas_brasil, left_on='cnpj', right_on='cnpj')

In [10]:
deputados_socios_empresas.to_csv('resultados/empresas_deputados.csv', index=False)

In [11]:
deputados_socios_empresas.to_excel('resultados/empresas_deputados.xlsx',sheet_name='Sheet1')

In [12]:
deputados_todos_socios_empresas = pd.merge(deputados_socios_empresas, socios_brasil, left_on='cnpj', right_on='cnpj')

In [13]:
deputados_todos_socios_empresas.to_csv('resultados/empresas_deputados_todos_socios.csv', index=False)

In [14]:
deputados_todos_socios_empresas.to_excel('resultados/empresas_deputados_todos_socios.xlsx',sheet_name='Sheet1')

Depois foi feita uma checagem manual de cada linha no site da Receita (http://www.receita.fazenda.gov.br/PessoaJuridica/CNPJ/cnpjreva/Cnpjreva_Solicitacao.asp) para confirmar dados e agregar mais informações

Le arquivo preenchido com informações do site da Receita (http://www.receita.fazenda.gov.br/PessoaJuridica/CNPJ/cnpjreva/Cnpjreva_Solicitacao.asp)

In [3]:
empresas_deputados = pd.read_excel('resultados/empresas_deputados_todos_socios.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x), 
                                                                                                                  'cpf': lambda x: str(x),  
                                                                                                                  'cnpj': lambda x: str(x),  
                                                                                                                  'documento': lambda x: str(x)})

In [4]:
def limpeza3(linha):
    documento = str(linha["cnpj"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
empresas_deputados['cnpj_raiz'] = empresas_deputados.apply(limpeza3, axis=1)

In [6]:
empresas_deputados.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5608 entries, 0.0 to 5649.0
Data columns (total 20 columns):
uf                         5604 non-null object
nome_completo              5608 non-null object
sequencial                 5604 non-null object
cpf                        5604 non-null object
nome_urna                  5604 non-null object
partido_eleicao            5604 non-null object
cnpj                       5608 non-null object
indicador                  5598 non-null float64
documento                  5598 non-null object
qualificacao               5598 non-null float64
nome_empresa               5608 non-null object
estado                     5608 non-null object
socios                     5608 non-null object
situacao_empresa           3154 non-null object
cidade                     3121 non-null object
cnae_principal             3160 non-null object
propriedade                5603 non-null object
capital_social             3122 non-null object
agricola_ou_relaciona

In [7]:
empresas_deputados = empresas_deputados[(empresas_deputados['propriedade'] == 'sim')]

In [25]:
empresas_deputados.to_excel('resultados/empresas_deputados_todostipos_ativas_baixadas.xlsx',sheet_name='Sheet1')

In [26]:
empresas_rurais = empresas_deputados[(empresas_deputados['situacao_empresa'] == 'ativa') & 
                                     (empresas_deputados['agricola_ou_relacionada'] == 'sim')]

In [27]:
empresas_rurais.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 173 entries, 9.0 to 5625.0
Data columns (total 20 columns):
uf                         171 non-null object
nome_completo              173 non-null object
sequencial                 171 non-null object
cpf                        171 non-null object
nome_urna                  171 non-null object
partido_eleicao            171 non-null object
cnpj                       173 non-null object
indicador                  171 non-null float64
documento                  171 non-null object
qualificacao               171 non-null float64
nome_empresa               173 non-null object
estado                     173 non-null object
socios                     173 non-null object
situacao_empresa           173 non-null object
cidade                     173 non-null object
cnae_principal             173 non-null object
propriedade                173 non-null object
capital_social             173 non-null object
agricola_ou_relacionada    173 non-null 

In [10]:
empresas_rurais.reset_index().head()

,index,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,cnpj,indicador,documento,...,nome_empresa,estado,socios,situacao_empresa,cidade,cnae_principal,propriedade,capital_social,agricola_ou_relacionada,cnpj_raiz
0,9.0,PA,JOSE BENITO PRIANTE JUNIOR,140000000456,17293618204,PRIANTE,PMDB,04322499000104,2.0,,...,AGROPECUARIA RIO DA PRATA LTDA - EPP,Pará,JOSE BENITO PRIANTE JUNIOR,ativa,Santa Isabel do Pará,01.33-4-01 - Cultivo de açaí,sim,750000,sim,04322499
1,10.0,PA,JOSE BENITO PRIANTE JUNIOR,140000000456,17293618204,PRIANTE,PMDB,04322499000104,2.0,,...,AGROPECUARIA RIO DA PRATA LTDA - EPP,Pará,JOSE BENITO PRIANTE NETO,ativa,Santa Isabel do Pará,01.33-4-01 - Cultivo de açaí,sim,750000,sim,04322499
2,11.0,PA,JOSE BENITO PRIANTE JUNIOR,140000000456,17293618204,PRIANTE,PMDB,04322499000104,2.0,,...,AGROPECUARIA RIO DA PRATA LTDA - EPP,Pará,LILIANE PEREIRA PINTO PRIANTE,ativa,Santa Isabel do Pará,01.33-4-01 - Cultivo de açaí,sim,750000,sim,04322499
3,13.0,PA,JOSE BENITO PRIANTE JUNIOR,140000000456,17293618204,PRIANTE,PMDB,20008829000148,2.0,,...,AGROINDUSTRIAL ACAI DA PRATA LTDA - ME,Pará,JOSE BENITO PRIANTE JUNIOR,ativa,Santa Isabel do Pará,01.33-4-01 - Cultivo de açaí,sim,50000,sim,20008829
4,14.0,PA,JOSE BENITO PRIANTE JUNIOR,140000000456,17293618204,PRIANTE,PMDB,20008829000148,2.0,,...,AGROINDUSTRIAL ACAI DA PRATA LTDA - ME,Pará,JOSE BENITO PRIANTE NETO,ativa,Santa Isabel do Pará,01.33-4-01 - Cultivo de açaí,sim,50000,sim,20008829


In [28]:
empresas_rurais.to_excel('resultados/empresas_rurais_ativas.xlsx',sheet_name='Sheet1')

In [11]:
a = len(empresas_rurais['cnpj'].value_counts())
a

60

# Checa se empresas rurais de deputados estão na lista de embargos do Ibama

In [12]:
ibama = pd.read_excel('/home/reinaldo/Documentos/Code/RB2/ibama/repo/embargos/ibama_14_7_18.xlsx',sheet_name='Planilha1')
ibama = ibama[pd.notnull(ibama['CPF_CNPJ'])]
ibama['Data_insercao_Lista'] = pd.to_datetime(ibama['Data_insercao_Lista'])

def limpeza(linha):
    documento = str(linha["CPF_CNPJ"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

ibama["CPF_CNPJ_limpo"] = ibama.apply(limpeza, axis=1)

In [13]:
def limpeza2(linha):
    documento = str(linha["CPF_CNPJ_limpo"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
ibama['cnpj_raiz'] = ibama.apply(limpeza2, axis=1)

In [14]:
ibama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59252 entries, 0 to 59693
Data columns (total 18 columns):
N_TAD                         59252 non-null int64
SERIE_TAD                     59252 non-null object
area_ha                       21036 non-null float64
N_AI                          56759 non-null float64
Nome_Razao_Social             59252 non-null object
CPF_CNPJ                      59252 non-null object
Localizacao_Imovel            52876 non-null object
UF_Embargo                    59252 non-null object
Municipio_Embargo             59252 non-null object
cod_IBGE_Municipio_Embargo    59252 non-null int64
Municipio                     59252 non-null object
Bairro                        57525 non-null object
Endereco                      59014 non-null object
Julgamento                    59252 non-null object
Infracao                      56758 non-null object
Data_insercao_Lista           59251 non-null datetime64[ns]
CPF_CNPJ_limpo                59252 non-null object

In [15]:
ibama_empresas_rurais = pd.merge(empresas_rurais.drop_duplicates('cnpj_raiz'), ibama.drop_duplicates('cnpj_raiz'), left_on='cnpj_raiz', right_on='cnpj_raiz')

In [16]:
ibama_empresas_rurais.nome_empresa

0    BRASFIBRA INDUSTRIA E COMERCIO DE CHAPAS DE MA...
1                                        AGROPEIXE S/A
Name: nome_empresa, dtype: object

In [29]:
ibama_empresas_rurais.to_excel('resultados/empresas_rurais_embargadas_ibama.xlsx',sheet_name='Sheet1')

# Checa se empresas rurais de deputados estão na lista do trabalho escravo

In [17]:
te = pd.read_csv("/home/reinaldo/Documentos/Code/RB2/te/repo/dados/lista_te_consolidada_junho_2018.csv",sep=';',encoding = 'latin_1', converters={'CNPJ_CPF_CEI_limpo': lambda x: str(x)})

In [18]:
te = te[pd.notnull(te['CNPJ_CPF_CEI'])]

In [19]:
# Preenche zeros a esquerda nos documentos
def compara1(linha):
    if len(linha['CNPJ_CPF_CEI_limpo']) < 14 and len(linha['CNPJ_CPF_CEI_limpo']) >= 12:
        conta = 14 - (len(linha['CNPJ_CPF_CEI_limpo']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['CNPJ_CPF_CEI_limpo']
    elif len(linha['CNPJ_CPF_CEI_limpo']) == 14:
        alterado = linha['CNPJ_CPF_CEI_limpo']
    elif len(linha['CNPJ_CPF_CEI_limpo']) < 11:
        conta = 11 - (len(linha['CNPJ_CPF_CEI_limpo']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['CNPJ_CPF_CEI_limpo']
    elif len(linha['CNPJ_CPF_CEI_limpo']) == 11:
        alterado = linha['CNPJ_CPF_CEI_limpo']
    
    return alterado

nova = te.apply(compara1, axis=1) #lê dataframe e chama a função

te['documento_inteiro'] = nova

def compara2(linha):
    documento = str(linha["documento_inteiro"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    


te['cnpj_raiz'] = te.apply(compara2, axis=1)

In [20]:
te.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5707 entries, 0 to 5830
Data columns (total 18 columns):
Data_lista            5707 non-null object
DATA_INCLUS           3123 non-null object
Proprietario          5707 non-null object
Nome_propriedade      5707 non-null object
Municipio             5691 non-null object
Estado                5699 non-null object
CNPJ_CPF_CEI          5707 non-null object
CNPJ_CPF_CEI_limpo    5707 non-null object
Trab_Envolv           4695 non-null float64
Ramo_atividade        2840 non-null object
Localizacao           2734 non-null object
Cod_ativ              3141 non-null object
Tipo_lista            5707 non-null object
Unnamed: 13           0 non-null float64
Unnamed: 14           0 non-null float64
Unnamed: 15           0 non-null float64
documento_inteiro     5707 non-null object
cnpj_raiz             5707 non-null object
dtypes: float64(4), object(14)
memory usage: 847.1+ KB


In [21]:
nome = te[(te['Nome_propriedade'] == 'AGROVAS-AGROPECUARIA VALE DO SUIA LTDA')]
nome

,Data_lista,DATA_INCLUS,Proprietario,Nome_propriedade,Municipio,Estado,CNPJ_CPF_CEI,CNPJ_CPF_CEI_limpo,Trab_Envolv,Ramo_atividade,Localizacao,Cod_ativ,Tipo_lista,Unnamed: 13,Unnamed: 14,Unnamed: 15,documento_inteiro,cnpj_raiz


In [22]:
te_empresas_rurais = pd.merge(empresas_rurais.drop_duplicates('cnpj_raiz'), te.drop_duplicates('cnpj_raiz'), left_on='cnpj_raiz', right_on='cnpj_raiz')

In [23]:
te_empresas_rurais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 37 columns):
uf                         1 non-null object
nome_completo              1 non-null object
sequencial                 1 non-null object
cpf                        1 non-null object
nome_urna                  1 non-null object
partido_eleicao            1 non-null object
cnpj                       1 non-null object
indicador                  1 non-null float64
documento                  1 non-null object
qualificacao               1 non-null float64
nome_empresa               1 non-null object
estado                     1 non-null object
socios                     1 non-null object
situacao_empresa           1 non-null object
cidade                     1 non-null object
cnae_principal             1 non-null object
propriedade                1 non-null object
capital_social             1 non-null object
agricola_ou_relacionada    1 non-null object
cnpj_raiz                  1 non-null obje

In [30]:
te_empresas_rurais.to_excel('resultados/empresas_rurais_na_lista_te.xlsx',sheet_name='Sheet1')